<a href="https://colab.research.google.com/github/Matlup45/Web-Scraping-projects/blob/main/Web_Scraping_Project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the necessary packages
!pip install selenium pandas
!apt-get update
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.7 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ub

In [2]:
# Import required libraries
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [3]:
# Set up Chrome options
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [6]:
# Initialize the WebDriver
driver = webdriver.Chrome(options=options)

base_url = "https://hprera.nic.in/PublicDashboard"
driver.get(base_url)


In [7]:
try:
    # Wait until the "Registered Projects" tab is loaded
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, 'reg-Projects'))
    ).click()
    print("Registered Projects tab clicked")

    # Wait until the projects are loaded
    WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div#reg-Projects div.shadow a[onclick*="tab_project_main_ApplicationPreview"]'))
    )
    print("Projects loaded")

    # Find project links
    project_links = driver.find_elements(By.CSS_SELECTOR, 'div#reg-Projects div.shadow a[onclick*="tab_project_main_ApplicationPreview"]')
    print(f"Found {len(project_links)} project links")

    projects = []

    # Process each project link (limit to first 6)
    for link in project_links[:6]:
        # Use JavaScript to click on the link
        try:
            driver.execute_script("arguments[0].click();", link)

            # Wait for the modal to be visible and loading spinner to disappear
            WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.ID, 'project-menu-html'))
            )
            WebDriverWait(driver, 30).until(
                EC.invisibility_of_element_located((By.CSS_SELECTOR, '.spinner-border'))
            )

            # Extract project details using XPath
            details = {}
            detail_elements = driver.find_elements(By.XPATH, '//*[@id="project-menu-html"]//table/tbody/tr')

            for element in detail_elements:
                try:
                    key = element.find_element(By.XPATH, './td[1]').text.strip()
                    value = element.find_element(By.XPATH, './td[2]').text.strip()
                    details[key] = value
                except Exception as e:
                    print(f"Error extracting details from element: {e}")

            print(f"Details fetched: {details}")
            projects.append(details)

            # Close the modal
            driver.find_element(By.CSS_SELECTOR, 'button.close').click()
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, 'reg-Projects'))
            )
        except Exception as e:
            print(f"Error processing link: {e}")
            continue

finally:
    # Close the WebDriver
    driver.quit()

Registered Projects tab clicked
Projects loaded
Found 196 project links
Details fetched: {'Name': 'MANAVINDER SINGH', 'Gender': 'Male', 'Father/Mother/Guardian Name': 'Lt. Sh. MUKHINDER SINGH', 'Photograph': '', 'Promoter Type': 'Individual', 'PAN No.': 'ACLPS2284H PAN Card', 'Mobile No.': '8527733503', 'Alternate Mobile No.': '-NA-', 'Email Id': 'manav@imperialholding.in', 'Alternate Email Id': 'bhawna.kapoor@imperialholding.in', 'Correspondence Address': 'Villette Kothi, Khalini, Shimla, Khalini, Shimla, Himachal Pradesh (171002) Address Proof', 'Permanent Address': 'Villette Kothi, Khalini, Shimla, Khalini, Shimla, Himachal Pradesh (171002) Address Proof', 'GSTIN No.': '-NA-', 'Authority Letter': '-NA-', 'Whether Himachali Agriculturist?': 'Yes'}
Details fetched: {'Name': 'MANAVINDER SINGH', 'Gender': 'Male', 'Father/Mother/Guardian Name': 'Lt. Sh. MUKHINDER SINGH', 'Photograph': '', 'Promoter Type': 'Individual', 'PAN No.': 'ACLPS2284H PAN Card', 'Mobile No.': '8527733503', 'Altern

In [8]:
# Check if we have projects
if projects:
    df = pd.DataFrame(projects)
    print(df)

    df.to_csv('registered_projects.csv', index=False)
else:
    print("No projects found or an error occurred.")

                                  Name Gender Father/Mother/Guardian Name  \
0                     MANAVINDER SINGH   Male     Lt. Sh. MUKHINDER SINGH   
1                     MANAVINDER SINGH   Male     Lt. Sh. MUKHINDER SINGH   
2                     MANAVINDER SINGH   Male     Lt. Sh. MUKHINDER SINGH   
3              UMA BAGOLIA AND D KONDA    NaN                         NaN   
4                      MS URBAN GREENS    NaN                         NaN   
5  M/S. JANTA LAND PROMOTERS PVT. LTD.    NaN                         NaN   

  Photograph   Promoter Type              PAN No.  Mobile No.  \
0                 Individual  ACLPS2284H PAN Card  8527733503   
1                 Individual  ACLPS2284H PAN Card  8527733503   
2                 Individual  ACLPS2284H PAN Card  8527733503   
3        NaN  Non-Individual  AFDPB7079J PAN File  9816245566   
4        NaN  Non-Individual  AAGFU9110M PAN File  9816188888   
5        NaN  Non-Individual  AABCJ3450D PAN File  8284922445   

  Al